# MLFlow Demo
Cette démo présente un cas d'usage de MLFlow. Il est divisé en deux
1. Entrenaiment simple
2. Exemple d'utilisation d'hyperopt

## Cas simple

### 1. Installation de libraires

In [2]:
!pip install mlflow matplotlib hyperopt

  Using cached dotenv-0.0.5.tar.gz (2.4 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [83 lines of output]
      /opt/conda/lib/python3.11/site-packages/setuptools/__init__.py:94: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
      !!
      
              ********************************************************************************
              Requirements should be satisfied by a PEP 517 installer.
              If you are using pip, you can try `pip install --use-pep517`.
              ********************************************************************************
      
      !!
        dist.fetch_build_eggs(dist.setup_requires)
        error: subprocess-exited-with-error
      
        × python setup.py egg_info did not run successfully.
        │ exit code: 1
        ╰─> [17 lines of output]
            Traceback (most recen

### 2. Configuration des variables d'environnement

In [ ]:
%env MLFLOW_TRACKING_SERVER_CERT_PATH=/home/jovyan/work/cert/ca.pem
%env MLFLOW_TRACKING_URI=https://public-tracking.mlflow-e00v0qa5n6ywddys75.backbone-e00m8trsn6mra9a7kk.msp.eu-north1.nebius.cloud
%env MLFLOW_TRACKING_USERNAME=mlops_course_admin
%env MLFLOW_TRACKING_PASSWORD=NAv2VcZafgku5D
%env MLFLOW_ENABLE_SYSTEM_METRICS_LOGGING=true

In [ ]:
# imports
import sys
import warnings
from itertools import cycle

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.linear_model import ElasticNet, enet_path
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

# Import mlflow
import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType
from mlflow.store.artifact.runs_artifact_repo import RunsArtifactRepository

# Import hyperopt
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

### 3. Définition de fonctions 

In [ ]:
# Définition des fonctions

# Load diabetes dataset
def load_data():
    diabetes = datasets.load_diabetes()
    X = diabetes.data
    y = diabetes.target
    
    # Create pandas DataFrame for sklearn ElasticNet linear_model
    Y = np.array([y]).transpose()
    d = np.concatenate((X, Y), axis=1)
    cols = diabetes.feature_names + ["progression"]
    data = pd.DataFrame(d, columns=cols)
    return data, X, y

# Evaluate metrics
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

# The training
def start_training(data_tuple, alpha=0.05, l1_ratio=0.05):
    data, X, y = data_tuple
    warnings.filterwarnings("ignore")
    np.random.seed(40)

    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)

    # The predicted column is "progression" which is a quantitative measure of disease progression one year after baseline
    train_x = train.drop(["progression"], axis=1)
    test_x = test.drop(["progression"], axis=1)
    train_y = train[["progression"]]
    test_y = test[["progression"]]

    # Run ElasticNet
    lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
    lr.fit(train_x, train_y)
    predicted_qualities = lr.predict(test_x)
    (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

    # Print out ElasticNet model metrics
    print(f"Elasticnet model (alpha={alpha:f}, l1_ratio={l1_ratio:f}):")
    print(f"  RMSE: {rmse}")
    print(f"  MAE: {mae}")
    print(f"  R2: {r2}")

    # Infer model signature
    predictions = lr.predict(train_x)
    signature = infer_signature(train_x, predictions)

    # Log mlflow attributes for mlflow UI
    mlflow.log_param("alpha", alpha)
    mlflow.log_param("l1_ratio", l1_ratio)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)
    mlflow.log_metric("mae", mae)
    mlflow.sklearn.log_model(lr, "model", signature=signature)

    # Compute paths
    eps = 5e-3  # the smaller it is the longer is the path

    print("Computing regularization path using the elastic net.")
    alphas_enet, coefs_enet, _ = enet_path(X, y, eps=eps, l1_ratio=l1_ratio)

    # Display results
    fig = plt.figure(1)
    ax = plt.gca()

    colors = cycle(["b", "r", "g", "c", "k"])
    neg_log_alphas_enet = -np.log10(alphas_enet)
    for coef_e, c in zip(coefs_enet, colors):
        l2 = plt.plot(neg_log_alphas_enet, coef_e, linestyle="--", c=c)

    plt.xlabel("-Log(alpha)")
    plt.ylabel("coefficients")
    title = "ElasticNet Path by alpha for l1_ratio = " + str(l1_ratio)
    plt.title(title)
    plt.axis("tight")

    # Save figures
    fig.savefig("ElasticNet-paths.png")

    # Close plot
    plt.close(fig)

    # Log artifacts (output files)
    mlflow.log_artifact("ElasticNet-paths.png")

### 4. Entrainement

In [ ]:
data_tuple = load_data()
start_training(data_tuple, 0.05, 0.05)
mlflow.end_run()

## Utilisation d'Hyperopt

In [ ]:
search_space=space = {
    'alpha': hp.choice('alpha', [0.0005,0.001, 0.01, 0.5, 1]),
    'l1_ratio': hp.choice('l1_ratio', [0.0005,0.001, 0.01, 0.5, 1]),
    'random_state': hp.choice('random_state', [0,7,42])
}

data, X, y = load_data()
warnings.filterwarnings("ignore")

train, test = train_test_split(data)
x_train = train.drop(["progression"], axis=1)
x_test = test.drop(["progression"], axis=1)
y_train = train[["progression"]]
y_test = test[["progression"]]

def objective (params):
    with mlflow.start_run(run_name=f"Run_{params['alpha']}_{params['l1_ratio']}_{params['random_state']}"):
        mlflow.set_tag("model", "elasticnet")
        mlflow.log_params(params)
        # Train Elasticnet
        lr = ElasticNet(**params)
        #evaluation = [( x_train, y_train), (x_test, y_test)]
        lr.fit(train_x, train_y)
        predicted_qualities = lr.predict(test_x)
        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)
        #accuracy=accuracy_score(y_test,y_pred)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)
        # Infer model signature
        predictions = lr.predict(train_x)
        signature = infer_signature(train_x, predictions)
        
        mlflow.sklearn.log_model(lr, "model", signature=signature)

    return {'loss': mae, 'status': STATUS_OK }

### Recherche de meilleures hyper params

In [ ]:
mlflow.set_experiment("mlflow-classification-experiment-2") 
best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=75,
    trials=Trials()
)